In [128]:
from lib.attr_ig import summarize_attributions
from lib.run_lime import get_xsum_data, init_model, tokenize_text
import torch
all_data = get_xsum_data()
device = 'cuda:0'
device = 'cpu'
device = torch.device(device)
mname = 'sshleifer/distilbart-cnn-6-6'
model, tokenizer = init_model(mname=mname, device=device)
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bert-base-srl-2020.03.24.tar.gz")

Checking /Users/jcxu/.cache/huggingface/datasets/a0c796d5dde60382897534854829044dab3d8f2cb3dddbbf1b7db89d7f21a61a.03dc4f0007c770026514ad44f62dcc3d2bcbbc3cb7004bfa1b166aefa790a994.py for additional imports.
Found main folder for dataset https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/xsum/xsum.py at /Users/jcxu/.cache/huggingface/modules/datasets_modules/datasets/xsum
Found specific version folder for dataset https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/xsum/xsum.py at /Users/jcxu/.cache/huggingface/modules/datasets_modules/datasets/xsum/128741c17b7a4c939dbf844a75a5e83deadd07deaf4b2eda2056ed8eebdb03ae
Found script file from https://raw.githubusercontent.com/huggingface/datasets/1.0.1/datasets/xsum/xsum.py to /Users/jcxu/.cache/huggingface/modules/datasets_modules/datasets/xsum/128741c17b7a4c939dbf844a75a5e83deadd07deaf4b2eda2056ed8eebdb03ae/xsum.py
Found dataset infos file from https://raw.githubusercontent.com/huggingface/datasets/1.0.1/da

INFO - Lock 140257389421008 acquired on /Users/jcxu/.cache/huggingface/datasets/a0c796d5dde60382897534854829044dab3d8f2cb3dddbbf1b7db89d7f21a61a.03dc4f0007c770026514ad44f62dcc3d2bcbbc3cb7004bfa1b166aefa790a994.py.lock
INFO - Lock 140257389421008 released on /Users/jcxu/.cache/huggingface/datasets/a0c796d5dde60382897534854829044dab3d8f2cb3dddbbf1b7db89d7f21a61a.03dc4f0007c770026514ad44f62dcc3d2bcbbc3cb7004bfa1b166aefa790a994.py.lock
INFO - Lock 140244101367120 acquired on /Users/jcxu/.cache/huggingface/datasets/_Users_jcxu_.cache_huggingface_datasets_xsum_default_1.1.0_128741c17b7a4c939dbf844a75a5e83deadd07deaf4b2eda2056ed8eebdb03ae.lock
INFO - Lock 140244101367120 released on /Users/jcxu/.cache/huggingface/datasets/_Users_jcxu_.cache_huggingface_datasets_xsum_default_1.1.0_128741c17b7a4c939dbf844a75a5e83deadd07deaf4b2eda2056ed8eebdb03ae.lock
INFO - Lock 140244101909864 acquired on /Users/jcxu/.cache/huggingface/datasets/_Users_jcxu_.cache_huggingface_datasets_xsum_default_1.1.0_128741c

In [129]:
data = all_data[0]
document, ref_sum = data['document'], data['summary']
token_ids, doc_str,_ = tokenize_text(tokenizer, document)

In [130]:
def forward_enc_dec_step(model, encoder_outputs, decoder_input_ids):
    # expanded_batch_idxs = (
    #         torch.arange(batch_size)
    #             .view(-1, 1)
    #             .repeat(1, 1)
    #             .view(-1)
    #             .to(device)
    #     )
    # encoder_outputs["last_hidden_state"] = encoder_outputs.last_hidden_state.index_select(
    #         0, expanded_batch_idxs
    #     )
    model_inputs = {"input_ids": None,
                        "past_key_values": None,
                        "encoder_outputs": encoder_outputs,
                        "decoder_input_ids": decoder_input_ids,
                        }
    outputs = model(**model_inputs, use_cache=False, return_dict=True,output_attentions=True)
    return outputs

In [131]:
from scipy.stats import entropy
def forward_step(model,encoder_outputs,  past_key_values, decoder_input_ids, inp_attn_mask=None):
    model_inputs = {"input_ids": None,
                        "past_key_values": past_key_values,
                        "attention_mask": inp_attn_mask,
                        "encoder_outputs": encoder_outputs,
                        "decoder_input_ids": decoder_input_ids,
                        }
    outputs = model(**model_inputs, use_cache=True, return_dict=True)
    next_token_logits = outputs.logits[:, -1, :]
    pred_distribution = torch.nn.functional.softmax(next_token_logits, dim=-1)
    numpy_pred_distb = pred_distribution.detach().numpy()
    ent = entropy(numpy_pred_distb,axis=-1)
    top5 = torch.topk(pred_distribution, 5, dim=-1, largest=True, sorted=True)
    next_token = torch.argmax(next_token_logits, dim=-1)
    next_token = next_token.unsqueeze(-1)
    cur_decoded = next_token.tolist()

    if "past_key_values" in outputs:
        past_key_values = outputs.past_key_values

    decoder_input_ids = torch.cat([decoder_input_ids, next_token], dim=-1)
    return ent, top5, cur_decoded,pred_distribution, past_key_values, decoder_input_ids


In [132]:
input_doc = token_ids['input_ids']
encoder_outputs = model.model.encoder(input_doc, return_dict=True)


In [133]:
from captum.attr import TokenReferenceBase

batch_size = input_doc.shape[0]
cur_len = 1
max_len = 6
has_eos = [False for _ in range(batch_size)]
bos_token_id = tokenizer.bos_token_id
eos_token_id = tokenizer.eos_token_id
decoded = [[bos_token_id] for _ in range(batch_size)]
decoder_input_ids = torch.LongTensor(decoded).to(device)
past_key_values = None
seq_length = input_doc.shape[1]
token_reference = TokenReferenceBase(reference_token_idx=tokenizer.pad_token_id)
reference_indice = token_reference.generate_reference(seq_length, device=device)
reference_indices = torch.stack([reference_indice for _ in range(batch_size)], dim=0)
# reference_indices[:, 0] = self.tokenizer.bos_token_id
# reference_indices[:, -1] = self.tokenizer.eos_token_id
ref_encoder_outputs = model.model.encoder(reference_indices, return_dict=True)

In [137]:
from lib.attr_ig import summarize_attributions,simple_viz_attribution
from torch.nn import CrossEntropyLoss
import logging
from typing import List

def fast_ig_enc_dec(decoded_inputs,tgt_class:int, encoder_outputs,ref_encoder_outputs ,device,num_steps = 51,):
    loss_fct = CrossEntropyLoss()
    interp_step_vec = ( encoder_outputs.last_hidden_state - ref_encoder_outputs.last_hidden_state) / num_steps
    ranges = torch.arange(1, num_steps + 1).unsqueeze(-1).unsqueeze(-1).to(device)
    repeated_raw = ranges * interp_step_vec
    interp_last_hidden_state = repeated_raw + ref_encoder_outputs.last_hidden_state
    interp_encoder_outputs = ref_encoder_outputs
    interp_encoder_outputs.last_hidden_state =interp_last_hidden_state

    # interp_decoded = [decoded_inputs for _ in range(num_steps)]
    interp_decoded = decoded_inputs.repeat(num_steps,1)
    interp_decoder_input_ids =torch.LongTensor(interp_decoded).to(device)
    with torch.enable_grad():
        interp_encoder_outputs.last_hidden_state.retain_grad()
        interp_out = forward_enc_dec_step(model,interp_encoder_outputs,interp_decoder_input_ids)
        logits = interp_out.logits[:,-1,:]
        target = torch.ones(num_steps,dtype=torch.long) * tgt_class

        loss = loss_fct(logits, target)
        loss.backward(retain_graph=True)
        logging.info(f"Loss: {loss.tolist()}")
        raw_grad = interp_encoder_outputs.last_hidden_state.grad

        # Approximate the integral using the trapezodal rule
        approx_grad = (raw_grad[:-1] + raw_grad[1:]) / 2
        # print(approx_grad.size())
        avg_grad = torch.mean(approx_grad, dim=0)  # input len, hdim
        # print(encoder_outputs.last_hidden_state.size())
        # print(ref_encoder_outputs.last_hidden_state.size())
        integrated_gradient = (encoder_outputs.last_hidden_state - ref_encoder_outputs.last_hidden_state[0]) * avg_grad  # seq_len, hdim
    return integrated_gradient
"""
attr_mode = True
num_steps = 51
loss_fct = CrossEntropyLoss()
interp_step_vec = ( encoder_outputs.last_hidden_state - ref_encoder_outputs.last_hidden_state) / num_steps
ranges = torch.arange(1, num_steps + 1).unsqueeze(-1).unsqueeze(-1).to(device)
repeated_raw = ranges * interp_step_vec
interp_last_hidden_state = repeated_raw + ref_encoder_outputs.last_hidden_state
interp_encoder_outputs = ref_encoder_outputs
interp_encoder_outputs.last_hidden_state =interp_last_hidden_state
interp_past_key_values=None
interp_decoded = [[bos_token_id, bos_token_id,bos_token_id] for _ in range(num_steps)]
interp_decoder_input_ids =torch.LongTensor(interp_decoded).to(device)
with torch.enable_grad():
    interp_encoder_outputs.last_hidden_state.retain_grad()
    interp_out = forward_enc_dec_step(model,interp_encoder_outputs,interp_decoder_input_ids)
    logits = interp_out.logits[:,-1,:]
    target = torch.ones(num_steps,dtype=torch.long)

    loss = loss_fct(logits, target)
    loss.backward(retain_graph=True)
    logging.info(f"Loss: {loss.tolist()}")
    raw_grad = interp_encoder_outputs.last_hidden_state.grad

    # Approximate the integral using the trapezodal rule
    approx_grad = (raw_grad[:-1] + raw_grad[1:]) / 2
    avg_grad = torch.mean(approx_grad, dim=0)  # input len, hdim
    integrated_gradient = (encoder_outputs.last_hidden_state - ref_encoder_outputs.last_hidden_state) * avg_grad  # seq_len, hdim
    print(integrated_gradient)
    # extracted_attribution = summarize_attributions(integrated_gradient)
    # viz = simple_viz_attribution(tokenizer, input_ids_tensor, extracted_attribution)

"""

In [136]:

# do linear interpolation of these embeddings
# interp_step_vec = (embed_input_tensor - embed_ref_tensor) / num_steps
# ranges = torch.arange(1, num_steps + 1).unsqueeze(-1).unsqueeze(-1).to(self.model.device)
# repeated_raw = ranges * interp_step_vec
# interpolated_tensor = repeated_raw + embed_ref_tensor

# run the model forward
# with torch.enable_grad():
#     interpolated_tensor.retain_grad()

all_entropy = []
all_topk = []
while cur_len < max_len and (not all(has_eos)):
    cur_len +=1
    logging.debug(f"Step: {cur_len}")
    last_decoder_input_ids =decoder_input_ids
    ent, top5, cur_decoded, pred_distribution,past_key_values, decoder_input_ids = forward_step(model,encoder_outputs,past_key_values,decoder_input_ids)
    all_entropy.append(ent[0])
    all_topk.append(top5)
    cur_decoded = [ cur_dec_token[0] for cur_dec_token in cur_decoded]
    # print(cur_decoded)
    for idx in range(batch_size):
        if cur_decoded[idx] == tokenizer.eos_token_id or cur_decoded[idx] == 479:
            has_eos[idx]=True
    # print(cur_decoded[0])
    # Attribution
    # print(last_decoder_input_ids)
    ig = fast_ig_enc_dec(decoded_inputs=last_decoder_input_ids,
                         tgt_class=cur_decoded[0],
                         encoder_outputs=encoder_outputs,
                         ref_encoder_outputs=ref_encoder_outputs,device=device)
    extracted_attribution = summarize_attributions(ig)

    # process for viz
    extracted_attribution = extracted_attribution.squeeze(0)
    input_doc = input_doc.squeeze(0)
    viz = simple_viz_attribution(tokenizer, input_doc, extracted_attribution)
    logging.info(viz)

decoder_input_ids = decoder_input_ids[:,1:] # REMOVE <s>
all_decoded_tokens = decoder_input_ids.tolist()
decoded_sents = [[ tokenizer.convert_ids_to_tokens(x) for x in s] for s in all_decoded_tokens]

In [124]:
import string
def align_bpe_and_words(word_tokens, bpe_tokens, tokenizer):
    # word_tokens are list of str, bpe_tokens are list of int
    cache_word = ""
    cache_bpe = ""
    map_from_bpe_to_word = [-1 for _ in range(len(bpe_tokens))]
    map_cursor_bpe = 0
    map_cursor_word = 0
    word_tokens = [w.translate({ord(c): None for c in string.whitespace}) for w in word_tokens]
    bpe_txts = [ tokenizer.decode(x).translate({ord(c): None for c in string.whitespace}) for x in bpe_tokens]
    # print(word_tokens)
    # print(bpe_txts)
    while len(word_tokens) > 0 and len(bpe_txts)>0:
        if len(cache_bpe) == len(cache_word):
            _pop_bpe = bpe_txts.pop(0)
            _pop_word = word_tokens.pop(0)
            cache_bpe += _pop_bpe
            cache_word += _pop_word
            map_from_bpe_to_word[map_cursor_bpe] = map_cursor_word
            map_cursor_bpe += 1
            if len(cache_word) == len(cache_bpe):
               map_cursor_word+=1
        elif len(cache_word)>len(cache_bpe):
            _pop_bpe = bpe_txts.pop(0)
            cache_bpe += _pop_bpe
            map_from_bpe_to_word[map_cursor_bpe] = map_cursor_word
            map_cursor_bpe += 1
        else:
            raise RuntimeError(f"{bpe_txts}\n{word_tokens}")
    # print(map_from_bpe_to_word)
    # print(word_tokens)
    # print(bpe_txts)
    return map_from_bpe_to_word

src = "ASSS ss as a :good'boy 'but' not real."
# src = "The SRL model was evaluated on the CoNLL 2012 dataset. "
srl_pred = predictor.predict(sentence=src)
srl_words = srl_pred['words']
codes = tokenizer.encode(src)
codes = codes[1:-1]
for c in codes:
    print(tokenizer.decode(c))
txts = tokenizer.convert_ids_to_tokens(codes)
align_bpe_and_words(srl_words,codes,tokenizer)

AS
SS
 ss
 as
 a
 :
good
'
boy
 '
but
'
 not
 real
.


[0, 0, 0, 1, 2, 3, 4, 4, 4, 4, 5, 6, 7, 8, 9]